### Importing the Libraries

In [4]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [5]:
tf.__version__ 

'2.11.0'

### Part 1 - Data Preprocessing

#### Generating images for the Training set

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#### Generating images for the Test set

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
!wget https://github.com/AvijitChowdhury/Sign-Language-To-Text-Conversion1/archive/refs/heads/main.zip



In [ ]:
y!unzip main.zip

### Creating the Training set

In [24]:
training_set = train_datagen.flow_from_directory('/content/Sign-Language-To-Text-Conversion1-main/dataSet/trainingData',                                
                                                 target_size = (128, 128),
                                                 batch_size = 10,
                                                 color_mode = 'grayscale',                                
                                                 class_mode = 'categorical')

Found 12845 images belonging to 27 classes.


In [13]:
test_set = test_datagen.flow_from_directory('/content/Sign-Language-To-Text-Conversion1-main/dataSet/testingData',
                                            target_size = (128, 128),                                  
                                            batch_size = 10,        
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 4268 images belonging to 27 classes.


# Part 2: Building CNN


# Initializing CNN

In [14]:
classifier = tf.keras.models.Sequential()

#Step 1: Convolution

In [15]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=3, 
                                     padding="same", 
                                     activation="relu", 
                                     input_shape=[128, 128, 1]))

#Step 2:  Pooling

In [16]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

# Adding a second convolution layer

In [17]:
classifier.add(tf.keras.layers.Conv2D(filters=32, 
                                      kernel_size=3, 
                                      padding="same", 
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

# Step 3: Flattering

In [18]:
classifier.add(tf.keras.layers.Flatten())

# Step 4: Full Connection

In [19]:
classifier.add(tf.keras.layers.Dense(units=128, 
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=27, activation='softmax')) # softmax for more than 2

### Part 3 - Training the CNN

#### Compiling the CNN

In [20]:
classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])

#### Training the CNN on the Training set and evaluating it on the Test set

In [21]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 32768)             0         
                                                                 
 dense (Dense)               (None, 128)               4

In [22]:
classifier.fit(training_set,
                  epochs = 5,
                  validation_data = test_set)

Epoch 1/5
1285/1285 [==============================] - 369s 286ms/step - loss: 2.2999 - accuracy: 0.2848 - val_loss: 0.6703 - val_accuracy: 0.7706
Epoch 2/5
1285/1285 [==============================] - 345s 269ms/step - loss: 1.2500 - accuracy: 0.5682 - val_loss: 0.3310 - val_accuracy: 0.8868
Epoch 3/5
1285/1285 [==============================] - 371s 289ms/step - loss: 0.9769 - accuracy: 0.6634 - val_loss: 0.2009 - val_accuracy: 0.9517
Epoch 4/5
1285/1285 [==============================] - 341s 265ms/step - loss: 0.7751 - accuracy: 0.7370 - val_loss: 0.1501 - val_accuracy: 0.9330
Epoch 5/5
1285/1285 [==============================] - 358s 278ms/step - loss: 0.6786 - accuracy: 0.7752 - val_loss: 0.1553 - val_accuracy: 0.9492


#### Saving the Model

In [23]:
model_json = classifier.to_json()
with open("model_new.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model_new.h5')
print('Weights saved')

Model Saved
Weights saved


In [26]:
classifier.fit(training_set,
                  epochs = 20,
                  validation_data = test_set)

Epoch 1/20
1285/1285 [==============================] - 360s 281ms/step - loss: 0.6161 - accuracy: 0.8027 - val_loss: 0.0803 - val_accuracy: 0.9787
Epoch 2/20
1285/1285 [==============================] - 343s 267ms/step - loss: 0.5278 - accuracy: 0.8274 - val_loss: 0.0637 - val_accuracy: 0.9864
Epoch 3/20
1285/1285 [==============================] - 357s 278ms/step - loss: 0.4837 - accuracy: 0.8445 - val_loss: 0.0498 - val_accuracy: 0.9869
Epoch 4/20
1285/1285 [==============================] - 356s 277ms/step - loss: 0.4631 - accuracy: 0.8501 - val_loss: 0.0476 - val_accuracy: 0.9899
Epoch 5/20
1285/1285 [==============================] - 355s 276ms/step - loss: 0.4192 - accuracy: 0.8649 - val_loss: 0.0355 - val_accuracy: 0.9913
Epoch 6/20
1285/1285 [==============================] - 353s 274ms/step - loss: 0.4112 - accuracy: 0.8659 - val_loss: 0.0501 - val_accuracy: 0.9810
Epoch 7/20
1285/1285 [==============================] - 352s 274ms/step - loss: 0.3660 - accuracy: 0.8819 - val_

In [31]:
model2_json = classifier.to_json()
with open("model2_new.json", "w") as json_file:
    json_file.write(model2_json)
print('Model Saved')
classifier.save_weights('model2_new.h5')
print('Weights saved')

Model Saved
Weights saved


In [ ]:
!cp '/content/model2_new.h5' '/content/drive/MyDrive'